In [1]:
import pandas as pd
import numpy as np
from multiprocessing import Pool
from functools import partial
from tqdm import tqdm_notebook as tqdm
from datetime import date, timedelta
import pickle
import Model_func as func

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, TimeDistributed, LeakyReLU, Conv2D, Concatenate, BatchNormalization, MaxPooling2D, AveragePooling1D, Reshape, Conv1D
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras import layers, optimizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [4]:
with open(r'D:\data\train_local\train_v1', 'rb') as fp:
    train = pickle.load(fp)
with open(r'D:\data\train_local\val_v1', 'rb') as fp:
    val = pickle.load(fp)
with open(r'D:\data\train_local\test_v1', 'rb') as fp:
    test = pickle.load(fp)

Transfer Data Form to trainable

Model Separating Asset and Trade

In [54]:
train_trade = np.array([[data[i].values.tolist() for i in range(2)] for data in train])
train_asset = np.array([[data[i].values.tolist() for i in range(2, 5)] for data in train])
train_demo = np.array([data[5] for data in train])
label_train = [data[7] for data in train]
Y_train = pd.DataFrame(label_train, columns=['label'])
Y_train = pd.get_dummies(Y_train, columns=['label'])
Y_train = Y_train.values

In [55]:
val_trade = np.array([[data[i].values.tolist() for i in range(2)] for data in val])
val_asset = np.array([[data[i].values.tolist() for i in range(2, 5)] for data in val])
val_demo = np.array([data[5] for data in val])
label_val = [data[7] for data in val]
Y_val = pd.DataFrame(label_val, columns=['label'])
Y_val = pd.get_dummies(Y_val, columns=['label'])
Y_val = Y_val.values

In [56]:
test_trade = np.array([[data[i].values.tolist() for i in range(2)] for data in test])
test_asset = np.array([[data[i].values.tolist() for i in range(2, 5)] for data in test])
test_demo = np.array([data[5] for data in test])
label_test = [data[7] for data in test]
Y_test = pd.DataFrame(label_test, columns=['label'])
Y_test = pd.get_dummies(Y_test, columns=['label'])
Y_test = Y_test.values

Model Separating Asset(different asset) and Trade(Buy and sell)

In [16]:
train_trade_B = np.array([data[0].values.tolist() for data in train])
train_trade_S = np.array([data[1].values.tolist() for data in train])
train_asset_ST = np.array([data[2].values.tolist() for data in train])
train_asset_MP = np.array([data[3].values.tolist() for data in train])
train_asset_SS = np.array([data[4].values.tolist() for data in train])
train_demo = np.array([data[5] for data in train])
label_train = [data[7] for data in train]
Y_train = pd.DataFrame(label_train, columns=['label'])
Y_train = pd.get_dummies(Y_train, columns=['label'])
Y_train = Y_train.values

In [17]:
val_trade_B = np.array([data[0].values.tolist() for data in val])
val_trade_S = np.array([data[1].values.tolist() for data in val])
val_asset_ST = np.array([data[2].values.tolist() for data in val])
val_asset_MP = np.array([data[3].values.tolist() for data in val])
val_asset_SS = np.array([data[4].values.tolist() for data in val])
val_demo = np.array([data[5] for data in val])
label_val = [data[7] for data in val]
Y_val = pd.DataFrame(label_val, columns=['label'])
Y_val = pd.get_dummies(Y_val, columns=['label'])
Y_val = Y_val.values

In [18]:
test_trade_B = np.array([data[0].values.tolist() for data in test])
test_trade_S = np.array([data[1].values.tolist() for data in test])
test_asset_ST = np.array([data[2].values.tolist() for data in test])
test_asset_MP = np.array([data[3].values.tolist() for data in test])
test_asset_SS = np.array([data[4].values.tolist() for data in test])
test_demo = np.array([data[5] for data in test])
label_test = [data[7] for data in test]
Y_test = pd.DataFrame(label_test, columns=['label'])
Y_test = pd.get_dummies(Y_test, columns=['label'])
Y_test = Y_test.values

In [61]:
train_trade.shape[1:]

(2, 120, 11)

In [64]:
input_trade = Input(train_trade.shape[1:])

Treat asset and trading data as image

In [66]:
input_trade = Input(train_trade.shape[1:])
trade_model = Conv2D(filters=5, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=(2, 2),
              data_format='channels_first')(input_trade)
trade_model = Dropout(0.2)(trade_model)
trade_model = Conv2D(filters=1, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=(5, 1),
              data_format='channels_first')(trade_model)
trade_model = Dropout(0.2)(trade_model)
trade_model = Flatten()(trade_model)
input_asset = Input(train_asset.shape[1:])
asset_model = Conv2D(filters=3, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=(5, 5),
              data_format='channels_first')(input_asset)
asset_model = Dropout(0.2)(asset_model)
asset_model = Conv2D(filters=1, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=(5, 1),
              data_format='channels_first')(asset_model)
asset_model = Dropout(0.2)(asset_model)
asset_model = Flatten()(asset_model)
input_demographic = Input(train_demo.shape[1:])
combinedInput = concatenate([trade_model, asset_model, input_demographic])
full_model = Dense(128, activation='relu')(combinedInput)
full_model = Dense(32, activation='relu')(full_model)
full_model = Dense(8, activation='relu')(full_model)
full_model = Dense(2, activation='softmax')(full_model)
model = Model([input_trade, input_asset, input_demographic], full_model)
opt = optimizers.Nadam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.AUC()])
model.fit([train_trade, train_asset, train_demo], Y_train, validation_data=([val_trade, val_asset, val_demo], Y_val), batch_size=8, epochs=10, verbose=2)

Train on 40655 samples, validate on 1967 samples
Epoch 1/10
40655/40655 - 26s - loss: 0.4439 - auc_9: 0.8751 - val_loss: 0.3930 - val_auc_9: 0.9042
Epoch 2/10
40655/40655 - 25s - loss: 0.4126 - auc_9: 0.8934 - val_loss: 0.3961 - val_auc_9: 0.9033
Epoch 3/10
40655/40655 - 25s - loss: 0.4015 - auc_9: 0.8993 - val_loss: 0.3952 - val_auc_9: 0.9030
Epoch 4/10
40655/40655 - 26s - loss: 0.3944 - auc_9: 0.9030 - val_loss: 0.3795 - val_auc_9: 0.9144
Epoch 5/10
40655/40655 - 25s - loss: 0.3850 - auc_9: 0.9077 - val_loss: 0.3733 - val_auc_9: 0.9149
Epoch 6/10
40655/40655 - 25s - loss: 0.3760 - auc_9: 0.9123 - val_loss: 0.3703 - val_auc_9: 0.9164
Epoch 7/10
40655/40655 - 26s - loss: 0.3713 - auc_9: 0.9145 - val_loss: 0.3776 - val_auc_9: 0.9112
Epoch 8/10
40655/40655 - 26s - loss: 0.3582 - auc_9: 0.9207 - val_loss: 0.4028 - val_auc_9: 0.9016
Epoch 9/10
40655/40655 - 26s - loss: 0.3485 - auc_9: 0.9251 - val_loss: 0.3858 - val_auc_9: 0.9072
Epoch 10/10
40655/40655 - 26s - loss: 0.3375 - auc_9: 0.9300

Treat trading and asset data as documents

In [62]:
input_trade_B = Input(train_trade_B.shape[1:])
input_trade_S = Input(train_trade_S.shape[1:])
trade_model_B = Conv1D(filters=3, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_trade_B)
trade_model_B = Flatten()(trade_model_B)
trade_model_S = Conv1D(filters=3, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_trade_S)
trade_model_S = Flatten()(trade_model_S)
trade_model = concatenate([trade_model_B, trade_model_S])
trade_model = Dense(128, activation='relu')(trade_model)
asset_model_ST = Conv1D(filters=1, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_asset_ST)
asset_model_ST = Flatten()(asset_model_ST)
asset_model_MP = Conv1D(filters=1, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_asset_MP)
asset_model_MP = Flatten()(asset_model_MP)
asset_model_SS = Conv1D(filters=1, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_asset_SS)
asset_model_SS = Flatten()(asset_model_SS)
trade_model = concatenate([trade_model_B, trade_model_S])
asset_model = concatenate([asset_model_ST, asset_model_MP, asset_model_SS])
combinedInput = concatenate([trade_model, asset_model])
full_model = Dense(128, activation='relu')(combinedInput)
full_model = Dense(32, activation='relu')(full_model)
full_model = Dense(8, activation='relu')(full_model)
full_model = Dense(2, activation='softmax')(full_model)
model = Model([input_trade_B, input_trade_S, input_asset_ST, input_asset_MP, input_asset_SS], trade_model)
opt = optimizers.Nadam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.AUC()])

In [34]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           [(None, 120, 11)]    0                                            
__________________________________________________________________________________________________
input_42 (InputLayer)           [(None, 120, 11)]    0                                            
__________________________________________________________________________________________________
conv1d_34 (Conv1D)              (None, 116, 3)       168         input_41[0][0]                   
__________________________________________________________________________________________________
conv1d_35 (Conv1D)              (None, 116, 3)       168         input_42[0][0]                   
____________________________________________________________________________________________

In [68]:
input_trade_B = Input(train_trade_B.shape[1:])
input_trade_S = Input(train_trade_S.shape[1:])
trade_model_B = Conv1D(filters=3, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_trade_B)
trade_model_B = Flatten()(trade_model_B)
trade_model_S = Conv1D(filters=3, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_trade_S)
trade_model_S = Flatten()(trade_model_S)
trade_model = Concatenate(axis=-1)([trade_model_B, trade_model_S])
input_asset_ST = Input(train_asset_ST.shape[1:])
input_asset_MP = Input(train_asset_MP.shape[1:])
input_asset_SS = Input(train_asset_SS.shape[1:])
asset_model_ST = Conv1D(filters=3, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_asset_ST)
asset_model_ST = Flatten()(asset_model_ST)
asset_model_MP = Conv1D(filters=3, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_asset_MP)
asset_model_MP = Flatten()(asset_model_MP)
asset_model_SS = Conv1D(filters=3, 
              strides=1, 
              padding='valid', 
              activation='relu',
              kernel_size=5)(input_asset_SS)
asset_model_SS = Flatten()(asset_model_SS)
asset_model = Concatenate(axis=-1)([asset_model_ST, asset_model_MP, asset_model_SS])
input_demographic = Input(train_demo.shape[1:])
combinedInput = Concatenate()([trade_model, asset_model, input_demographic])
full_model = Dense(128, activation='relu')(combinedInput)
full_model = Dense(32, activation='relu')(full_model)
full_model = Dense(8, activation='relu')(full_model)
full_model = Dense(2, activation='softmax')(full_model)
model = Model([input_trade_B, input_trade_S, input_asset_ST, input_asset_MP, input_asset_SS, input_demographic], full_model)
opt = optimizers.Nadam(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=[tf.keras.metrics.AUC()])
model.fit([train_trade_B, train_trade_S, train_asset_ST, train_asset_MP, train_asset_SS, train_demo], Y_train, validation_data=([val_trade_B, val_trade_S, val_asset_ST, val_asset_MP, val_asset_SS, val_demo], Y_val), batch_size=8, epochs=5, verbose=2)

Train on 40655 samples, validate on 1967 samples
Epoch 1/5
40655/40655 - 29s - loss: 0.4261 - auc_10: 0.8858 - val_loss: 0.3792 - val_auc_10: 0.9111
Epoch 2/5
40655/40655 - 28s - loss: 0.3695 - auc_10: 0.9153 - val_loss: 0.3463 - val_auc_10: 0.9258
Epoch 3/5
40655/40655 - 28s - loss: 0.3451 - auc_10: 0.9264 - val_loss: 0.3411 - val_auc_10: 0.9299
Epoch 4/5
40655/40655 - 29s - loss: 0.3330 - auc_10: 0.9315 - val_loss: 0.3514 - val_auc_10: 0.9263
Epoch 5/5
40655/40655 - 28s - loss: 0.3219 - auc_10: 0.9359 - val_loss: 0.3427 - val_auc_10: 0.9286


Evaluation

In [69]:
result = func.Evaluation([test_trade_B, test_trade_S, test_asset_ST, test_asset_MP, test_asset_SS, test_demo], label_test, threshold=0.5, model=model)
print(result[0])
print(result[1])

0.8572778827977315
              precision    recall  f1-score   support

      ACTIVE       0.90      0.80      0.84       514
       CHURN       0.83      0.91      0.87       544

    accuracy                           0.86      1058
   macro avg       0.86      0.86      0.86      1058
weighted avg       0.86      0.86      0.86      1058



In [67]:
result = func.Evaluation([test_trade, test_asset, test_demo], label_test, threshold=0.5, model=model)
print(result[0])
print(result[1])

0.8393194706994329
              precision    recall  f1-score   support

      ACTIVE       0.85      0.82      0.83       514
       CHURN       0.83      0.86      0.85       544

    accuracy                           0.84      1058
   macro avg       0.84      0.84      0.84      1058
weighted avg       0.84      0.84      0.84      1058



In [75]:
model.save(r'D:\Customer_Value\model\first_stage_1D.h5')

Evaluation on other samples

In [3]:
model = load_model(r'D:\Customer_Value\model\first_stage.h5')

W0303 09:03:22.524970 12480 deprecation.py:506] From C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0303 09:03:22.527948 12480 deprecation.py:506] From C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0303 09:03:22.528949 12480 deprecation.py:506] From C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\pyth

Evaluation on another sample with 100% active data

In [70]:
with open(r'D:\data\evaluate\local\evaluate_large', 'rb') as fp:
    eval_test = pickle.load(fp)

In [73]:
# eval_trade_B = np.array([data[0].values.tolist() for data in eval_test])
# eval_trade_S = np.array([data[1].values.tolist() for data in eval_test])
# eval_asset_ST = np.array([data[2].values.tolist() for data in eval_test])
# eval_asset_MP = np.array([data[3].values.tolist() for data in eval_test])
# eval_asset_SS = np.array([data[4].values.tolist() for data in eval_test])
eval_demo = np.array([data[5] for data in eval_test])
label_test_eval = [data[7] for data in eval_test]
Y_test_eval = pd.DataFrame(label_test_eval, columns=['label'])
Y_test_eval = pd.get_dummies(Y_test_eval, columns=['label'])
Y_test_eval = Y_test_eval.values

In [7]:
# eval_test_trade = np.array([[data[i].values.tolist() for i in range(2)] for data in eval_test])
# eval_test_asset = np.array([[data[i].values.tolist() for i in range(2, 5)] for data in eval_test])
eval_test_demo = np.array([data[5] for data in eval_test])
label_test_eval = [data[7] for data in eval_test]
Y_test_eval = pd.DataFrame(label_test_eval, columns=['label'])
Y_test_eval = pd.get_dummies(Y_test_eval, columns=['label'])
Y_test_eval = Y_test_eval.values

In [43]:
test_trade.shape

(1041, 2, 120, 11)

In [10]:
result = func.Evaluation([eval_test_trade, eval_test_asset, eval_test_demo], label_test_eval, model, threshold=0.5)
print(result[0])
print(result[1])

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.83064
              precision    recall  f1-score   support

      ACTIVE       1.00      0.83      0.91    150000
       CHURN       0.00      0.00      0.00         0

    accuracy                           0.83    150000
   macro avg       0.50      0.42      0.45    150000
weighted avg       1.00      0.83      0.91    150000



In [74]:
result = func.Evaluation([eval_trade_B, eval_trade_S, eval_asset_ST, eval_asset_MP, eval_asset_SS, eval_demo], label_test_eval, threshold=0.5, model=model)
print(result[0])
print(result[1])

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.7871333333333334
              precision    recall  f1-score   support

      ACTIVE       1.00      0.79      0.88    150000
       CHURN       0.00      0.00      0.00         0

    accuracy                           0.79    150000
   macro avg       0.50      0.39      0.44    150000
weighted avg       1.00      0.79      0.88    150000

